<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 3 Assignment 1*

# Recurrent Neural Networks and Long Short Term Memory (LSTM)

![Monkey at a typewriter](https://upload.wikimedia.org/wikipedia/commons/thumb/3/3c/Chimpanzee_seated_at_typewriter.jpg/603px-Chimpanzee_seated_at_typewriter.jpg)

It is said that [infinite monkeys typing for an infinite amount of time](https://en.wikipedia.org/wiki/Infinite_monkey_theorem) will eventually type, among other things, the complete works of Wiliam Shakespeare. Let's see if we can get there a bit faster, with the power of Recurrent Neural Networks and LSTM.

This text file contains the complete works of Shakespeare: https://www.gutenberg.org/files/100/100-0.txt

Use it as training data for an RNN - you can keep it simple and train character level, and that is suggested as an initial approach.

Then, use that trained RNN to generate Shakespearean-ish text. Your goal - a function that can take, as an argument, the size of text (e.g. number of characters or lines) to generate, and returns generated text of that size.

Note -  Shakespeare wrote an awful lot. It's OK, especially initially, to sample/use smaller data and parameters, so you can have a tighter feedback loop when you're trying to get things running. Then, once you've got a proof of concept - start pushing it more!

In [0]:
"""
- Use it as training data for an (RNN character level)
- Generate shakspearean text
- Create a function that takes size (# of characters to generate) and returns text of that size
"""
import requests

response = requests.get('https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')
text = response.text

In [2]:
len(text)

1115394

In [3]:
type(text)

str

In [4]:
# Create a set of unique characters
chars = list(set(text))

# First 5
chars[:5]

['a', 'Q', '&', 'J', 'h']

In [0]:
# Lookup tables 
char_int = {c:i for i, c in enumerate(chars)}
int_char = {i:c for i, c in enumerate(chars)}

In [6]:
""" Sequence the data """

# Size of character sequences
max_len = 40

# Number of characters to advance to to start a new sequence
step = 5

# Entire text encoded as numbers representing characters
encoded = [char_int[c] for c in text]

sequences = [] # Holds the sequences
next_char = []


for i in range(0, len(encoded) - max_len, step):
  """
  Loop 0 to the length of the text - 40. Increment by the step (5)
  """
  # Encode slices of the text in 40 character sequences
  sequences.append(encoded[i : i + max_len])

  # Holding the cut off's of these sequences
  next_char.append(encoded[i + max_len])

print('sequences: ', len(sequences))

sequences:  223071


In [0]:
"""
len(sequences) = 223071
max_len = 40
len(chars) = 65
"""
import numpy as np

# Creates a matrix of Falses of shape 223071, 40, 65
x = np.zeros((len(sequences), max_len, len(chars)), dtype=np.bool)

# Creates a matrix of Falses of shape 223071, 1, 65
y = np.zeros((len(sequences),  len(chars)), dtype=np.bool)


# Loop through the entire text
for i, sequence in enumerate(sequences):

    # loop through a sequence
    for t, char in enumerate(sequence):

        # encode the char as true (rest stay false)
        x[i,t,char] = 1
        
    # encode the target chars     
    y[i, next_char[i]] = 1

In [85]:
from tensorflow.keras.callbacks import LambdaCallback, EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from tensorflow.keras.optimizers import RMSprop
import numpy as np

import random
import sys
import os

import pandas as pd
from __future__ import print_function

# Build the model
model = Sequential()

# 40 true false matrices of one hot encoded characters
model.add(LSTM(65, dropout=0.2, recurrent_dropout=0.2, input_shape=(max_len, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')

In [0]:
# HELPER FUNCTIONS
def sample(preds):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / 1
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    
    print()
    print('----- Generating text after Epoch: %d' % epoch)
    
    start_index = random.randint(0, len(text) - max_len - 1)
    
    generated = ''
    
    sentence = text[start_index: start_index + max_len]
    generated += sentence
    
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)
    
    for i in range(400):
        x_pred = np.zeros((1, max_len, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_int[char]] = 1
            
        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds)
        next_char = int_char[next_index]
        
        sentence = sentence[1:] + next_char
        
        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()


print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

In [87]:
# fit the model
model.fit(x, y,
          batch_size=256,
          epochs=10,
          callbacks=[print_callback])

Epoch 1/10
872/872 [==============================] - ETA: 0s - loss: 2.8632
----- Generating text after Epoch: 0
----- Generating with seed: "u desperate pilot, now at once run on
Th"
u desperate pilot, now at once run on
Thinus so, marse oi bek en dhal. loan ththe, Irt benlm age shecis tlunFe to ryritme h;umil, yoh g att thise'is no, te theth ntWet oy on' wilmof amice

Threst Wer memtairs 
nete?

M
OTIGL ICN: r tour
wPwinhe
nK ou the Ix mate li filws Ilsth fonrt thepan,
eoe wyais cowatasath, bur,
Te yed menonAeg eemer3eondon tI 'at: miBagh cegue hore.
Aosk yod ionfer, hem bIrpkedashrun siligenemor,iM swiI basu, Shod
872/872 [==============================] - 136s 156ms/step - loss: 2.8632
Epoch 2/10
872/872 [==============================] - ETA: 0s - loss: 2.4890
----- Generating text after Epoch: 1
----- Generating with seed: " Summon the town.

MARCIUS:
How far off "
 Summon the town.

MARCIUS:
How far off une.

NANTITHII FiI:
Cf hags bowthes hor clot, lnote farrt sereWki, Anr ond

In [0]:
# Funtion to create N characters of text

def Generator(N):

  # Get a random starting text index
  start_index = random.randint(0, len(text) - max_len - 1)

  
  # Get a sequence from the random index to 40 on
  sentence = text[start_index: start_index + max_len]

  # Predict N characters and append to 'predicted' string
  generated = ''
  generated += sentence
  chars_to_predict = N + len(sentence)

  
  # For loop matching size of characters N
  for i in range(chars_to_predict):

    # Create a list of falses of size 1, 
    x_pred = np.zeros((1, max_len, len(chars)))
    for t, char in enumerate(sentence):
        x_pred[0, t, char_int[char]] = 1
        
    preds = model.predict(x_pred, verbose=0)[0]

    next_index = sample(preds)
    generated_char = int_char[next_index]

    generated += generated_char

  generated = generated[40:]

  return generated

In [0]:
Generator(2000)

In [49]:
  # # Get a random starting text index
  # start_index = random.randint(0, len(text) - max_len - 1)

  
  # # Get a sequence from the random index to 40 on
  # sentence = text[start_index: start_index + max_len]

  # # for t, char in enumerate(sentence):
  # #   print(t, ' :',char)

  # # x_pred = np.zeros((1, max_len, len(chars)))
  # # x_pred[0][0]

  # # For loop matching size of characters N
  # for i in range(3):

  #   # Create a list of falses of size 1, 
  #   x_pred = np.zeros((1, max_len, len(chars)))
  #   for t, char in enumerate(sentence):
  #       x_pred[0, t, char_int[char]] = 1
  #       print(t, ' : ', char)

  # # len(x_pred[0][0])

0  :  H
1  :  e
2  :   
3  :  l
4  :  i
5  :  t
6  :  t
7  :  l
8  :  e
9  :   
10  :  t
11  :  h
12  :  o
13  :  u
14  :  g
15  :  h
16  :  t
17  :   
18  :  o
19  :  f
20  :   
21  :  t
22  :  h
23  :  i
24  :  s
25  :   
26  :  d
27  :  i
28  :  v
29  :  i
30  :  d
31  :  e
32  :  d
33  :   
34  :  f
35  :  r
36  :  i
37  :  e
38  :  n
39  :  d
0  :  H
1  :  e
2  :   
3  :  l
4  :  i
5  :  t
6  :  t
7  :  l
8  :  e
9  :   
10  :  t
11  :  h
12  :  o
13  :  u
14  :  g
15  :  h
16  :  t
17  :   
18  :  o
19  :  f
20  :   
21  :  t
22  :  h
23  :  i
24  :  s
25  :   
26  :  d
27  :  i
28  :  v
29  :  i
30  :  d
31  :  e
32  :  d
33  :   
34  :  f
35  :  r
36  :  i
37  :  e
38  :  n
39  :  d
0  :  H
1  :  e
2  :   
3  :  l
4  :  i
5  :  t
6  :  t
7  :  l
8  :  e
9  :   
10  :  t
11  :  h
12  :  o
13  :  u
14  :  g
15  :  h
16  :  t
17  :   
18  :  o
19  :  f
20  :   
21  :  t
22  :  h
23  :  i
24  :  s
25  :   
26  :  d
27  :  i
28  :  v
29  :  i
30  :  d
31  :  e
32  :  d
33  :   
34  

# Resources and Stretch Goals

## Stretch goals:
- Refine the training and generation of text to be able to ask for different genres/styles of Shakespearean text (e.g. plays versus sonnets)
- Train a classification model that takes text and returns which work of Shakespeare it is most likely to be from
- Make it more performant! Many possible routes here - lean on Keras, optimize the code, and/or use more resources (AWS, etc.)
- Revisit the news example from class, and improve it - use categories or tags to refine the model/generation, or train a news classifier
- Run on bigger, better data

## Resources:
- [The Unreasonable Effectiveness of Recurrent Neural Networks](https://karpathy.github.io/2015/05/21/rnn-effectiveness/) - a seminal writeup demonstrating a simple but effective character-level NLP RNN
- [Simple NumPy implementation of RNN](https://github.com/JY-Yoon/RNN-Implementation-using-NumPy/blob/master/RNN%20Implementation%20using%20NumPy.ipynb) - Python 3 version of the code from "Unreasonable Effectiveness"
- [TensorFlow RNN Tutorial](https://github.com/tensorflow/models/tree/master/tutorials/rnn) - code for training a RNN on the Penn Tree Bank language dataset
- [4 part tutorial on RNN](http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/) - relates RNN to the vanishing gradient problem, and provides example implementation
- [RNN training tips and tricks](https://github.com/karpathy/char-rnn#tips-and-tricks) - some rules of thumb for parameterizing and training your RNN